In [1]:
import pandas as pd
import datetime
# from datetime import timedelta

In [2]:
구별_도로면적_2017 = pd.read_excel('데이터/구별_도로면적_2017.xls')
구별_도로면적_2018 = pd.read_excel('데이터/구별_도로면적_2018.xls')

In [3]:
구별_도로면적_2018_포장면적 = 구별_도로면적_2018.iloc[3:][['기간', '자치구','합계.3']].reset_index(drop=True)
구별_도로면적_2018_포장면적 = 구별_도로면적_2018_포장면적.rename(columns={'기간':'년','자치구':'구이름','합계.3':'포장도로면적'})
구별_도로면적_2018_포장면적['년']=구별_도로면적_2018_포장면적['년'].astype(int)

In [4]:
구별_도로면적_2018_포장면적.head()

,년,구이름,포장도로면적
0,2018,종로구,2688271
1,2018,중구,1876189
2,2018,용산구,3150277
3,2018,성동구,3024659
4,2018,광진구,3396817


In [5]:
제설제_투입량 = pd.read_csv('../../data/데이터정리/미완성_투입자재.csv')

In [6]:
제설제_투입량.날짜 = 제설제_투입량.날짜.map(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))

In [7]:
제설제_투입량_2018 = 제설제_투입량[(datetime.datetime(2018,1,1)<=제설제_투입량.날짜) & (제설제_투입량.날짜<=datetime.datetime(2018,12,31))].reset_index(drop=True)

In [8]:
# 제설제_투입량_2018.groupby(['날짜', '년', '월', '일', '구이름', '재설자재']).sum()['투입자재량'].reset_index()
제설제_투입량_2018_group = 제설제_투입량_2018.groupby(['년', '구이름']).sum()['투입자재량'].reset_index()

In [9]:
제설제_투입량_2018_group = 제설제_투입량_2018_group[제설제_투입량_2018_group['구이름'].isin(구별_도로면적_2018_포장면적.구이름.unique().tolist())]
제설제_투입량_2018_group = 제설제_투입량_2018_group.reset_index(drop=True)

In [10]:
제설제_투입량_2018_group.head()

,년,구이름,투입자재량
0,2018,강남구,2140.0
1,2018,강동구,1356.0
2,2018,강북구,2025.0
3,2018,강서구,780.0
4,2018,관악구,1050.0


In [11]:
df = pd.merge(구별_도로면적_2018_포장면적,제설제_투입량_2018_group, how='outer')
df.head()

,년,구이름,포장도로면적,투입자재량
0,2018,종로구,2688271,1496.0
1,2018,중구,1876189,1294.0
2,2018,용산구,3150277,1589.0
3,2018,성동구,3024659,792.0
4,2018,광진구,3396817,801.0


In [12]:
df['면적대비투입정도'] = df['투입자재량']/df['포장도로면적']
df.head()

,년,구이름,포장도로면적,투입자재량,면적대비투입정도
0,2018,종로구,2688271,1496.0,0.000556492
1,2018,중구,1876189,1294.0,0.000689696
2,2018,용산구,3150277,1589.0,0.0005044
3,2018,성동구,3024659,792.0,0.000261848
4,2018,광진구,3396817,801.0,0.000235809


In [13]:
import folium
import json

In [14]:
geo_path = '데이터/02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [15]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                tiles='stamentoner')
map.choropleth(geo_data=geo_str)
map.save('result/sample_map_seoul.html')
map

/home/jongsky/anaconda3/envs/python_3/lib/python3.7/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [16]:
df=df.set_index('구이름')

In [17]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                tiles='stamentoner')
map.choropleth(geo_data=geo_str,
              data=df['면적대비투입정도'],
              columns=[df.index, df['면적대비투입정도']],
              fill_color='PuRd',
              key_on='feature.id')

map.save('result/sample_map_snow.html')
map

In [18]:
df

,년,포장도로면적,투입자재량,면적대비투입정도
구이름,,,,
종로구,2018,2688271,1496.0,0.000556492
중구,2018,1876189,1294.0,0.000689696
용산구,2018,3150277,1589.0,0.0005044
성동구,2018,3024659,792.0,0.000261848
광진구,2018,3396817,801.0,0.000235809
동대문구,2018,3144310,728.0,0.000231529
중랑구,2018,2938827,1750.0,0.000595476
성북구,2018,4014836,1477.0,0.000367886
강북구,2018,1974677,2025.0,0.00102548
